In [1]:
# Imports
import chromadb
from chromadb.utils import embedding_functions
from pathlib import Path
import textwrap

In [2]:
# ---------------- SETTINGS ----------------
CHROMA_DB_PATH = Path("..") / "output" / "chroma_store"
COLLECTION_NAME = "card_docs"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
N_RESULTS = 5
# ------------------------------------------

In [3]:
# Connect to Chroma
client = chromadb.PersistentClient(path=str(CHROMA_DB_PATH))

In [4]:
# Load collection with embedding function
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL_NAME
)
collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_fn
)

In [5]:
def query_chroma_with_scores(question: str, n_results: int = 5, width: int = 100):
    """
    Query Chroma and return top N chunks with metadata, wrapped text, and similarity score.
    """
    results = collection.query(
        query_texts=[question],
        n_results=n_results,
        include=["metadatas", "documents", "distances"]  # include distances/scores
    )

    wrapped_output = ""
    for idx, doc in enumerate(results['documents'][0]):
        metadata = results['metadatas'][0][idx]
        score = results['distances'][0][idx]  # similarity distance
        # Convert distance to a more intuitive "score" (optional)
        relevance_score = 1 - score  # higher = more relevant

        header = (
            f"[Card: {metadata['card']} | Date: {metadata['date']} | "
            f"File: {metadata['filename']} | Page: {metadata['page']} | "
            f"Chunk: {metadata['chunk_index']} | Score: {relevance_score:.3f}]"
        )
        wrapped_text = textwrap.fill(doc, width=width)
        wrapped_output += f"{header}\n{wrapped_text}\n\n"

    return wrapped_output.strip()


In [6]:
question = "What is the grocery cashback for DiscoverIt?"
answer = query_chroma_with_scores(question)
print(answer)  # print first 2000 chars


[Card: Discover | Date: 20250930 | File: Cashback.pdf | Page: 1 | Chunk: 0 | Score: 0.584]
Discover it® Cash Back Credit Card Unlimited Cashback Match Get an unlimited dollar-for-dollar match
of all the cash back you earn at the end of your first year, automatically. There is no limit to how
much we'll match.1 Earn 5% cash back on everyday purchases at different places you shop each
quarter, up to the quarterly maximum when you activate. Plus, earn 1% cash back on all other
purchases. Applies to all categories: Purchases must be made with merchants in the U.S. To qualify
for 5%, the purchase transaction date must be before or on the last day of the offer or promotion.
For online purchases, the transaction date from the merchant may be the date when the item ships.
Rewards are added to your account within two billing periods. Even if a purchase appears to fit in a
5% category, the merchant may not have a merchant category code (MCC) in that category. Merchants
and payment processors are

In [7]:
question = "Which card gives the highest rewards for dining out?"
answer = query_chroma_with_scores(question)
print(answer)  # print first 2000 chars


[Card: BOA Allegiant | Date: 20250930 | File: Cashback.pdf | Page: 1 | Chunk: 0 | Score: 0.482]
Allways Rewards Visa® card 40,000 Bonus Points Online Offer - equal to $400 off future Allegiant
travel when you redeem points APPLY NOW GET A RESPONSE IN AS FEW AS 30 SECONDS! APPLY HERE. OFFERS
VARY ELSEWHERE. 4.3 (3602) Voted Best Airline Credit Card by USA Today Readers Earn Points on Every
Purchase 3 points per $1 spent on Allegiant purchases (air, hotel, and car rental) 2 points per $1
spent on qualifying dining purchases 1 point per $1 spent APPLY NOW on all other purchases

[Card: Amex | Date: 20250930 | File: Cashback.pdf | Page: 8 | Chunk: 1 | Score: 0.453]
include fees or interest charges, balance transfers, cash advances, purchases of travelers checks,
purchases or reloading of prepaid cards, purchases of other cash equivalents, gift cards, person-to-
person transactions, or any portion of a purchase that is covered by Reward Dollars at point of
sale. Basic Card Members will earn

In [8]:
question = "Are there any restrictions on using the cashback at Amazon in Discover?"
answer = query_chroma_with_scores(question)
print(answer)  # print first 2000 chars


[Card: Discover | Date: 20250930 | File: Cashback.pdf | Page: 1 | Chunk: 0 | Score: 0.573]
Discover it® Cash Back Credit Card Unlimited Cashback Match Get an unlimited dollar-for-dollar match
of all the cash back you earn at the end of your first year, automatically. There is no limit to how
much we'll match.1 Earn 5% cash back on everyday purchases at different places you shop each
quarter, up to the quarterly maximum when you activate. Plus, earn 1% cash back on all other
purchases. Applies to all categories: Purchases must be made with merchants in the U.S. To qualify
for 5%, the purchase transaction date must be before or on the last day of the offer or promotion.
For online purchases, the transaction date from the merchant may be the date when the item ships.
Rewards are added to your account within two billing periods. Even if a purchase appears to fit in a
5% category, the merchant may not have a merchant category code (MCC) in that category. Merchants
and payment processors are